In [ ]:
pip install datasets

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor
from torch.nn.utils.rnn import pad_sequence

In [ ]:
# Defining the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads),
            num_layers
        )
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)  # 
        x = self.fc(x)
        return x


In [ ]:
# Loading the Iris dataset
iris_data = load_iris()
data = iris_data.data
targets = iris_data.target

In [ ]:
# Dividing the training set and test set
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, random_state=42)

In [ ]:
# Defining custom data sets
class IrisDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.long)
        return x, y

# Set random seeds
torch.manual_seed(42)

# Creating a data loader
train_dataset = IrisDataset(train_data, train_targets)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = IrisDataset(test_data, test_targets)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# Define hyperparameters
input_dim = 4
hidden_dim = 128
output_dim = 3
num_layers = 2
num_heads = 4
batch_size = 16
num_epochs = 50
learning_rate = 0.001

# Create models and optimizers
model = TransformerModel(input_dim, hidden_dim, output_dim, num_layers, num_heads)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training Model
for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        
        # Adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        total_samples += targets.size(0)
        
        loss.backward()
        optimizer.step()
        
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * total_correct / total_samples
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch 1/50, Loss: 0.1879, Accuracy: 93.33%
Epoch 2/50, Loss: 0.1023, Accuracy: 96.67%
Epoch 3/50, Loss: 0.0827, Accuracy: 97.50%
Epoch 4/50, Loss: 0.0795, Accuracy: 98.33%
Epoch 5/50, Loss: 0.0729, Accuracy: 96.67%
Epoch 6/50, Loss: 0.1071, Accuracy: 95.83%
Epoch 7/50, Loss: 0.0689, Accuracy: 99.17%
Epoch 8/50, Loss: 0.0876, Accuracy: 97.50%
Epoch 9/50, Loss: 0.1423, Accuracy: 96.67%
Epoch 10/50, Loss: 0.0745, Accuracy: 98.33%
Epoch 11/50, Loss: 0.1522, Accuracy: 95.83%
Epoch 12/50, Loss: 0.2950, Accuracy: 90.00%
Epoch 13/50, Loss: 0.1472, Accuracy: 95.00%
Epoch 14/50, Loss: 0.1250, Accuracy: 95.83%
Epoch 15/50, Loss: 0.1000, Accuracy: 95.83%
Epoch 16/50, Loss: 0.0987, Accuracy: 95.83%
Epoch 17/50, Loss: 0.1451, Accuracy: 94.17%
Epoch 18/50, Loss: 0.1432, Accuracy: 95.83%
Epoch 19/50, Loss: 0.1006, Accuracy: 96.67%
Epoch 20/50, Loss: 0.1141, Accuracy: 94.17%
Epoch 21/50, Loss: 0.1302, Accuracy: 95.00%
Epoch 22/50, Loss: 0.2392, Accuracy: 90.83%
Epoch 23/50, Loss: 0.2243, Accuracy: 90.8

In [ ]:
# Evaluating the model on a test set
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        # Adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 80.00%


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 检查训练集中的标签值范围
train_labels = [sample['label'] for sample in dataset['train']]
min_label = min(train_labels)
max_label = max(train_labels)
print("Min Label:", min_label)
print("Max Label:", max_label)

# 检查验证集中的标签值范围
val_labels = [sample['label'] for sample in dataset['test']]
min_label = min(val_labels)
max_label = max(val_labels)
print("Min Label:", min_label)
print("Max Label:", max_label)


Min Label: 0
Max Label: 199
Min Label: 0
Max Label: 199


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from transformers import ViTModel

# 加载数据集
from datasets import load_dataset
dataset = load_dataset("alkzar90/CC6204-Hackaton-Cub-Dataset")

from torchvision.transforms import ToTensor
from PIL import Image

# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        label = sample['label']

        # Resize the image while maintaining the aspect ratio
        image = self.resize_image(image, size=224)

        image = self.transform(image)

        return image, label

    @staticmethod
    def resize_image(image, size):
        # Resize the image while maintaining the aspect ratio
        width, height = image.size
        aspect_ratio = width / height
        if width > height:
            new_height = size
            new_width = int(new_height * aspect_ratio)
        else:
            new_width = size
            new_height = int(new_width / aspect_ratio)

        resized_image = image.resize((new_width, new_height), Image.BILINEAR)

        # Pad the image if necessary to achieve the desired size
        padded_image = Image.new("RGB", (size, size))
        padded_image.paste(resized_image, ((size - new_width) // 2, (size - new_height) // 2))

        return padded_image


# 划分训练集和验证集
transform = ToTensor()
train_dataset = CustomDataset(dataset['train'], transform)
val_dataset = CustomDataset(dataset['test'], transform)

# 创建数据加载器
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# 定义Transformer模型
class ImageTransformer(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ImageTransformer, self).__init__()
        self.transformer = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = self.transformer(x)
        x = x.last_hidden_state[:, 0, :]  # 提取CLS向量
        x = self.fc(x)
        return x

# 创建模型实例
input_dim = 768  # ViT模型的输出维度
num_classes = 200  # 分类的类别数
model = ImageTransformer(input_dim, num_classes)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    for batch in train_loader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = train_correct / train_total

    # Evaluate on the validation set
    model.eval()
    with torch.no_grad():
        val_correct = 0
        val_total = 0
        for batch in val_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

        val_accuracy = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch [1/10], Training Accuracy: 0.0032, Validation Accuracy: 0.0050
Epoch [2/10], Training Accuracy: 0.0038, Validation Accuracy: 0.0052
Epoch [3/10], Training Accuracy: 0.0027, Validation Accuracy: 0.0052
Epoch [4/10], Training Accuracy: 0.0047, Validation Accuracy: 0.0052
Epoch [5/10], Training Accuracy: 0.0043, Validation Accuracy: 0.0052
Epoch [6/10], Training Accuracy: 0.0042, Validation Accuracy: 0.0052
Epoch [7/10], Training Accuracy: 0.0057, Validation Accuracy: 0.0052
Epoch [8/10], Training Accuracy: 0.0042, Validation Accuracy: 0.0048
Epoch [9/10], Training Accuracy: 0.0037, Validation Accuracy: 0.0050
Epoch [10/10], Training Accuracy: 0.0047, Validation Accuracy: 0.0052
